In [1]:
%env JAVA_HOME=/usr/lib/jvm/java-11-openjdk-11.0.25.0.9-2.el8.x86_64
%env PATH=/storage/home/yqf5148/work/anaconda3/envs/volvopennstate-env/bin:storage/icds/swst/deployed/production/20220813/apps/anaconda3/2021.05_gcc-8.5.0/bin:/usr/lib/jvm/java-11-openjdk-11.0.25.0.9-2.el8.x86_64/bin/java:/usr/local/bin:/usr/bin:/usr/local/sbin:/usr/sbin:/storage/icds/tools/bin:/storage/sys/slurm/bin:/storage/icds/tools/bin:/storage/sys/slurm/bin:/storage/icds/tools/bin:/storage/sys/slurm/bin:/storage/icds/tools/bin:/storage/sys/slurm/bin:/storage/icds/tools/bin:/storage/sys/slurm/bin

env: JAVA_HOME=/usr/lib/jvm/java-11-openjdk-11.0.25.0.9-2.el8.x86_64
env: PATH=/storage/home/yqf5148/work/anaconda3/envs/volvopennstate-env/bin:storage/icds/swst/deployed/production/20220813/apps/anaconda3/2021.05_gcc-8.5.0/bin:/usr/lib/jvm/java-11-openjdk-11.0.25.0.9-2.el8.x86_64/bin/java:/usr/local/bin:/usr/bin:/usr/local/sbin:/usr/sbin:/storage/icds/tools/bin:/storage/sys/slurm/bin:/storage/icds/tools/bin:/storage/sys/slurm/bin:/storage/icds/tools/bin:/storage/sys/slurm/bin:/storage/icds/tools/bin:/storage/sys/slurm/bin:/storage/icds/tools/bin:/storage/sys/slurm/bin


In [2]:
import findspark
import pandas as pd
import numpy as np

import pyspark as psk
from pyspark import SparkConf, SparkContext
from pyspark.sql import SparkSession
from pyspark.sql.types import *
from pyspark.sql import functions as f
from pyspark.sql import Row
from pyspark.sql.types import StructType, StructField, StringType
from pyspark.sql import DataFrame
from pyspark.sql.functions import *

import time as t 
from datetime import date, datetime, timedelta
import joblib
from joblib import Parallel, delayed

import multiprocessing
from functools import reduce  
from math import modf
import sqlite3
import os
import sys

from delta import * 
from delta.tables import *
from delta import configure_spark_with_delta_pip
# import covalent as ct

import subprocess
import re
import random

from sklearn.preprocessing import OneHotEncoder
from sklearn.decomposition import PCA
# import umap.umap_ as umap
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score

import matplotlib.pyplot as plt

findspark.init()
findspark.find()

os.environ['PYSPARK_PYTHON'] = sys.executable
os.environ['PYSPARK_DRIVER_PYTHON'] = sys.executable

# os.environ['PYDEVD_DISABLE_FILE_VALIDATION']=1

spark = SparkSession.builder \
       .config("spark.ui.port", "4050") \
       .master("local[2]") \
       .appName("MyApp") \
       .config("spark.driver.maxResultSize", "20g")\
       .config("spark.driver.memory", "100g")\
       .getOrCreate()
sc = spark.sparkContext

# sqlContext = SQLContext(sc)


#both works
# 1: 
# spark.sql("set spark.sql.legacy.timeParserPolicy=LEGACY")    #To resolve the error for p1075_38 to_timestamp formating: You may get a different result due to the upgrading to Spark >= 3.0: Fail to parse '1/2/2019 20:40:00' in the new parser. You can set spark.sql.legacy.timeParserPolicy to LEGACY to restore the behavior before Spark 3.0, or set to CORRECTED and treat it as an invalid datetime string.
# Set Spark SQL legacy time parser policy to LEGACY to handle older date formats
# 2:
spark.conf.set("spark.sql.legacy.timeParserPolicy", "LEGACY")
# Increase the max fields in the string representation of a plan
spark.conf.set("spark.sql.execution.arrow.pyspark.enabled", "false")
spark.conf.set("spark.sql.debug.maxToStringFields", 2000)

ps: /storage/icds/RISE/sw8/anaconda/anaconda3/envs/tensorflow/lib/libuuid.so.1: no version information available (required by /usr/lib64/libblkid.so.1)
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/04/09 00:35:47 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
spark.read.option("header",True) \
          .csv("/storage/home/yqf5148/work/volvoPennState/data/dataset/VINs_data.csv") \
          .createOrReplaceTempView("VINs_data")

spark.read.option("header",True) \
          .csv("/storage/home/yqf5148/work/volvoPennState/PopulationWithChassisId.csv") \
          .createOrReplaceTempView("population")

In [4]:
df_all_pop = spark.sql('select * from population')
df_all_pop_p = df_all_pop.toPandas()

In [5]:
# Get all column names
all_columns = df_all_pop.columns

# Print the first 10
print("First 10 column names:")
for col in all_columns[:11]:
    print(col)


First 10 column names:
VIN
CHASSIS_ID
ENGINE_ASSEMB_DATE
ENGINE_ASSEMB_MONTH
VEH_ASSEMB_DATE
VEH_ASSEMB_MONTH
INS_DATE
INS_MONTH
ENGINE_SIZE
ENGINE_HP
VEH_TYPE


In [6]:
# Get all column names
all_columns = df_all_pop.columns

vin_column = [all_columns[0]]

# Include column 2 (index 1)
extra_column = [all_columns[2]]

# Pick columns from index 8 to the end
main_columns = all_columns[8:]

# Combine them
selected_columns = vin_column + extra_column + main_columns

# # Print how many columns are selected
# print(f"Number of selected columns: {len(selected_columns)}")

# # Print their names
# print("Selected column names:")
# for col in selected_columns:
#     print(col)


In [7]:
def evaluate_pca_single(df, categorical_columns, non_kola_columns, k=10):
    print("Step 1: Dropping NA rows...")
    df_filtered = df[non_kola_columns + categorical_columns].dropna()
    print(f"    Filtered shape: {df_filtered.shape}")

    print("Step 2: One-hot encoding KOLA features...")
    encoder = OneHotEncoder(sparse_output=False, handle_unknown='ignore')
    kola_encoded = encoder.fit_transform(df_filtered[categorical_columns])
    print(f"    Encoded shape: {kola_encoded.shape}")

    print("\nApplying PCA with 6 components...")
    pca = PCA(n_components=6)
    pca_result = pca.fit_transform(kola_encoded)

In [8]:
vin_column = "VIN"
df = df_all_pop_p.copy()

# Assuming df is your full dataset and selected_columns is already defined
kola_columns = [col for col in selected_columns if "KOLA" in col]
non_kola_columns = [col for col in selected_columns if col not in kola_columns or col == vin_column]

evaluate_pca_single(df, categorical_columns=kola_columns, non_kola_columns=non_kola_columns, k=10)

Step 1: Dropping NA rows...
    Filtered shape: (321746, 593)
Step 2: One-hot encoding KOLA features...
    Encoded shape: (321746, 7845)

Applying PCA with 6 components...


In [9]:
def evaluate_feature_separability(df, categorical_columns, non_kola_columns, vin_column="VIN", k=10):
    print("Step 1: Dropping NA rows...")
    df_filtered = df[non_kola_columns + categorical_columns].dropna()
    print(f"    Filtered shape: {df_filtered.shape}")

    print("Step 2: One-hot encoding KOLA features...")
    encoder = OneHotEncoder(sparse_output=False, handle_unknown='ignore')
    kola_encoded = encoder.fit_transform(df_filtered[categorical_columns])
    print(f"    Encoded shape: {kola_encoded.shape}")

    print("Step 3: Applying PCA with 6 components...")
    pca = PCA(n_components=6)
    pca_result = pca.fit_transform(kola_encoded)

    df_pca = pd.DataFrame(pca_result, columns=[f"PCA_{i+1}" for i in range(pca_result.shape[1])], index=df_filtered.index)
    df_combined = pd.concat([df_filtered[non_kola_columns], df_pca], axis=1)
    df_combined = df_combined.drop(columns=[vin_column])  # Drop VIN before clustering

    print("\nStep 4: Evaluating separability of each feature...")

    scores = {}
    for feature in df_combined.columns:
        feature_data = df_combined[[feature]].values
        kmeans = KMeans(n_clusters=k, random_state=42)
        labels = kmeans.fit_predict(feature_data)
        try:
            score = silhouette_score(feature_data, labels)
        except ValueError:
            score = -1  # If only one label found, silhouette score can't be calculated
        scores[feature] = score
        print(f"    Feature: {feature}, Silhouette Score: {score:.4f}")

    # Find the best scoring feature(s)
    best_score = max(scores.values())
    best_features = [f for f, s in scores.items() if s == best_score]
    print(f"\nBest feature(s): {best_features} with Silhouette Score: {best_score:.4f}")

    # Optional: Plot
    plt.figure(figsize=(10, 6))
    plt.bar(scores.keys(), scores.values(), color='skyblue')
    plt.xticks(rotation=45, ha='right')
    plt.ylabel("Silhouette Score")
    plt.title("Separability of Features Using KMeans Clustering")
    plt.tight_layout()
    plt.savefig("feature_separability_scores.pdf")
    plt.show()

    return best_features


In [10]:
best_features = evaluate_feature_separability(
    df=df_all_pop_p,
    categorical_columns=kola_columns,
    non_kola_columns=non_kola_columns,
    vin_column="VIN",
    k=10
)


Step 1: Dropping NA rows...
    Filtered shape: (321746, 593)
Step 2: One-hot encoding KOLA features...
    Encoded shape: (321746, 7845)
Step 3: Applying PCA with 6 components...

Step 4: Evaluating separability of each feature...


ValueError: could not convert string to float: '2014-11-10'